# XWAS/COLOC using GTEX v8 brain dataset

**Start date:** 2023-12-28

**Updated date:** 2023-12-28

**Author(s):** Ruth Chia

**Working directory on biowulf:** `/data/ALS_50k/DementiaSeq.TopmedJointCalled.June2020/LBD/Analysis.XWAS_GLM`

In [1]:
!pwd

/vf/users/ALS_50k/DementiaSeq.TopmedJointCalled.June2020/LBD/Analysis.XWAS_GLM


## ***UPDATED 12-28-2023:*** COLOC with GTEX v8 brain dataset

Some notes:
1. Downloaded 13 brain eQTL associations from GTEX v8. Location of formatted eQTL summ stats for 13 brain regions are: `/data/NDRS_LNG/GTEX_v8/GTEX_v8_chrX_brain/eQTL/Brain_*.v8.EUR.allpairs.chrX.txt`


2. The 13 brain regions are:
- `Brain_Amygdala`
- `Brain_Anterior_cingulate_cortex_BA24`
- `Brain_Caudate_basal_ganglia`
- `Brain_Cerebellar_Hemisphere`
- `Brain_Cerebellum`
- `Brain_Cortex`
- `Brain_Frontal_Cortex_BA9`
- `Brain_Hippocampus`
- `Brain_Hypothalamus`
- `Brain_Nucleus_accumbens_basal_ganglia`
- `Brain_Putamen_basal_ganglia`
- `Brain_Spinal_cord_cervical_c-1`
- `Brain_Substantia_nigra`

    
3. XWAS summary stats to run coloc on are (only for regions of interest with significant/subsignificant (significance threshold was set at 7.94x10-6) hit +/- 0.5Mb i.e. according the the coloc recommendation on `https://chr1swallace.github.io/coloc/articles/a02_data.html`):
    - females (not conditioned on ApoE): `/data/ALS_50k/DementiaSeq.TopmedJointCalled.June2020/LBD/Analysis.XWAS_GLM/females_only_x-autosomal-pc/toMeta.LBD.controls.UNRELATED.females.maf0.01overall.hg38.chrX.rsid_REFORMATTED.txt`
    - females (conditioned on ApoE): `/data/ALS_50k/DementiaSeq.TopmedJointCalled.June2020/LBD/Analysis.XWAS_GLM/cond_ApoE4_x-autosomal-pc/females_only/toMeta.LBD.controls.UNRELATED.females.maf0.01overall.hg38.chrX.rsid_REFORMATTED.txt`
    
____

**What I would need to do:**
1. Filter summ stats to include significant/subsignificant hit from summ stats - then flank out by 1Mb from top index variant.
2. Harmonize eQTL summ stats to include the same set of variants and to make sure that the variant ID is the same (can keep it to chr:pos:ref:alt' format.
3. Prep not summ stats for XWAS and eQTL so that it contains the necessary columns to run coloc using the coloc R package (tutorial to run coloc on two summ stats can be found: `https://chr1swallace.github.io/coloc/articles/a02_data.html`).

In [5]:
!mkdir Analysis.COLOC_redo

In [6]:
!mkdir Analysis.COLOC_redo/females_only
!mkdir Analysis.COLOC_redo/females_only_cond_ApoE4

In [7]:
!mkdir Analysis.COLOC_redo/GTEXv8_eQTL_brains
!mkdir Analysis.COLOC_redo/GTEXv8_sQTL_brains

### Create subset of snp region to run coloc on 

#### females-only xwas (unconditioned on APOE4)

top index variants from Table 1 of paper are:
- rs141773145 (pos = 19,513,849)
- rs12860838 (pos = 117,373,825)

use these variants and flank out by 1Mb

In [142]:
import pandas as pd
import numpy as np

females_uncond = pd.read_csv("/data/ALS_50k/DementiaSeq.TopmedJointCalled.June2020/LBD/Analysis.XWAS_GLM/females_only_x-autosomal-pc/toMeta.LBD.controls.UNRELATED.females.maf0.01overall.hg38.chrX.rsid_REFORMATTED.txt",sep="\t")
print(females_uncond.columns)

Index(['CHROM', 'POS', 'ID', 'rsID', 'EffectAllele', 'OtherAllele', 'BETA',
       'BETA_SE', 'OR', 'OR_low95', 'OR_high95', 'P', 'MAF_EffectAllele',
       'MAF_EffectAllele_CASE', 'MAF_EffectAllele_CTRL', 'CT_EffectAllele',
       'CT_EffectAllele_CASE', 'CT_EffectAllele_CTRL', 'ALLELE_CT',
       'CASE_ALLELE_CT', 'CTRL_ALLELE_CT'],
      dtype='object')


In [143]:
females_uncond.describe().T

,count,mean,std,min,25%,50%,75%,max
POS,258247.0,7.933355e+07,4.563753e+07,2.781514e+06,3.614308e+07,8.358717e+07,1.193624e+08,1.557006e+08
BETA,258247.0,-6.831485e-03,1.435296e-01,-1.369221e+00,-6.981106e-02,-2.739750e-03,6.471038e-02,9.655455e-01
BETA_SE,258247.0,1.180516e-01,6.720031e-02,5.539100e-02,6.369070e-02,8.736010e-02,1.600105e-01,4.408130e-01
OR,258247.0,1.003327e+00,1.435414e-01,2.543050e-01,9.325705e-01,9.972740e-01,1.066850e+00,2.626220e+00
OR_low95,258247.0,8.030415e-01,1.415592e-01,1.081022e-01,7.350166e-01,8.360580e-01,8.983707e-01,1.628731e+00
OR_high95,258247.0,1.275058e+00,2.717356e-01,5.841018e-01,1.111900e+00,1.193951e+00,1.344726e+00,4.234603e+00
P,258247.0,4.850853e-01,2.933628e-01,3.436790e-05,2.249820e-01,4.805790e-01,7.400285e-01,9.999950e-01
MAF_EffectAllele,258247.0,1.689946e-01,1.479782e-01,1.013960e-02,3.407580e-02,1.228390e-01,2.847410e-01,5.000000e-01
MAF_EffectAllele_CASE,258247.0,1.685472e-01,1.480764e-01,3.164560e-03,3.375530e-02,1.228900e-01,2.850740e-01,5.290080e-01
MAF_EffectAllele_CTRL,258247.0,1.692005e-01,1.480586e-01,7.038830e-03,3.398060e-02,1.223300e-01,2.839810e-01,5.128640e-01


In [144]:
females_uncond.head(2)

,CHROM,POS,ID,rsID,EffectAllele,OtherAllele,BETA,BETA_SE,OR,OR_low95,...,P,MAF_EffectAllele,MAF_EffectAllele_CASE,MAF_EffectAllele_CTRL,CT_EffectAllele,CT_EffectAllele_CASE,CT_EffectAllele_CTRL,ALLELE_CT,CASE_ALLELE_CT,CTRL_ALLELE_CT
0,X,2781514,chrX:2781514:C:A,rs311165,A,C,-0.025560,0.058254,0.974764,0.869586,...,0.660837,0.416057,0.410338,0.418689,2503,778,1725,6016,1896,4120
1,X,2781604,chrX:2781604:G:T,rs28579419,T,G,0.022065,0.082083,1.022310,0.870387,...,0.788093,0.138630,0.143460,0.136408,834,272,562,6016,1896,4120


In [145]:
females_uncond_indexvar_1_flank1Mb = females_uncond[(females_uncond.POS > 19513849 - 1000000) & (females_uncond.POS < 19513849 + 1000000)]
females_uncond_indexvar_2_flank1Mb = females_uncond[(females_uncond.POS > 117373825 - 1000000) & (females_uncond.POS < 117373825 + 1000000)]

In [146]:
females_uncond_indexvar_1_flank1Mb.to_csv("Analysis.COLOC_redo/females_only/XWAS_summ_stats_rs141773145_flank1Mb_no-maf-filter.txt",sep="\t",index=False,header=True)
females_uncond_indexvar_2_flank1Mb.to_csv("Analysis.COLOC_redo/females_only/XWAS_summ_stats_rs12860838_flank1Mb_no-maf-filter.txt",sep="\t",index=False,header=True)

In [147]:
print(females_uncond_indexvar_1_flank1Mb.shape)
print(females_uncond_indexvar_2_flank1Mb.shape)

(2419, 21)
(5314, 21)


#### females-only xwas (conditioned on APOE4)

top index variants from Table 1 of paper are:
- rs141773145(pos = 19,513,849)
- rs6648060 (pos = 76,575,769)
- rs141193614 (pos = 98,584,345)
- rs12860838 (pos = 117,373,825)

use these variants and flank out by 0.5Mb

In [149]:
import pandas as pd
import numpy as np

females_cond = pd.read_csv("/data/ALS_50k/DementiaSeq.TopmedJointCalled.June2020/LBD/Analysis.XWAS_GLM/cond_ApoE4_x-autosomal-pc/females_only/toMeta.LBD.controls.UNRELATED.females.maf0.01overall.hg38.chrX.rsid_REFORMATTED.txt",sep="\t")
print(females_cond.columns)

Index(['CHROM', 'POS', 'ID', 'rsID', 'EffectAllele', 'OtherAllele', 'BETA',
       'BETA_SE', 'OR', 'OR_low95', 'OR_high95', 'P', 'MAF_EffectAllele',
       'MAF_EffectAllele_CASE', 'MAF_EffectAllele_CTRL', 'CT_EffectAllele',
       'CT_EffectAllele_CASE', 'CT_EffectAllele_CTRL', 'ALLELE_CT',
       'CASE_ALLELE_CT', 'CTRL_ALLELE_CT'],
      dtype='object')


In [150]:
females_cond_indexvar_1_flank1Mb = females_cond[(females_cond.POS > 19513849 - 1000000) & (females_cond.POS < 19513849 + 1000000)]
females_cond_indexvar_2_flank1Mb = females_cond[(females_cond.POS > 76575769 - 1000000) & (females_cond.POS < 76575769 + 1000000)]
females_cond_indexvar_3_flank1Mb = females_cond[(females_cond.POS > 98584345 - 1000000) & (females_cond.POS < 98584345 + 1000000)]
females_cond_indexvar_4_flank1Mb = females_cond[(females_cond.POS > 117373825 - 1000000) & (females_cond.POS < 117373825 + 1000000)]

In [151]:
females_cond_indexvar_1_flank1Mb.to_csv("Analysis.COLOC_redo/females_only_cond_ApoE4/XWAS_summ_stats_rs141773145_flank1Mb_no-maf-filter.txt",sep="\t",index=False,header=True)
females_cond_indexvar_2_flank1Mb.to_csv("Analysis.COLOC_redo/females_only_cond_ApoE4/XWAS_summ_stats_rs6648060_flank1Mb_no-maf-filter.txt",sep="\t",index=False,header=True)
females_cond_indexvar_3_flank1Mb.to_csv("Analysis.COLOC_redo/females_only_cond_ApoE4/XWAS_summ_stats_rs141193614_flank1Mb_no-maf-filter.txt",sep="\t",index=False,header=True)
females_cond_indexvar_4_flank1Mb.to_csv("Analysis.COLOC_redo/females_only_cond_ApoE4/XWAS_summ_stats_rs12860838_flank1Mb_no-maf-filter.txt",sep="\t",index=False,header=True)

In [152]:
print(females_cond_indexvar_1_flank1Mb.shape)
print(females_cond_indexvar_2_flank1Mb.shape)
print(females_cond_indexvar_3_flank1Mb.shape)
print(females_cond_indexvar_4_flank1Mb.shape)

(2422, 21)
(5192, 21)
(2416, 21)
(5315, 21)


### Harmonize eQTL dataset

Use the unconditioned XWAS from females only to reformat variant ID.

The conditioned XWAS summ stats should have the same REF/ALT designation.

In [153]:
import pandas as pd
import numpy as np

xwas = pd.read_csv("/data/ALS_50k/DementiaSeq.TopmedJointCalled.June2020/LBD/Analysis.XWAS_GLM/females_only_x-autosomal-pc/toMeta.LBD.controls.UNRELATED.females.maf0.01overall.hg38.chrX.rsid_REFORMATTED.txt",sep="\t")     
print(xwas.shape)

(258247, 21)


In [154]:
xwas.head()

,CHROM,POS,ID,rsID,EffectAllele,OtherAllele,BETA,BETA_SE,OR,OR_low95,...,P,MAF_EffectAllele,MAF_EffectAllele_CASE,MAF_EffectAllele_CTRL,CT_EffectAllele,CT_EffectAllele_CASE,CT_EffectAllele_CTRL,ALLELE_CT,CASE_ALLELE_CT,CTRL_ALLELE_CT
0,X,2781514,chrX:2781514:C:A,rs311165,A,C,-0.025560,0.058254,0.974764,0.869586,...,0.660837,0.416057,0.410338,0.418689,2503,778,1725,6016,1896,4120
1,X,2781604,chrX:2781604:G:T,rs28579419,T,G,0.022065,0.082083,1.022310,0.870387,...,0.788093,0.138630,0.143460,0.136408,834,272,562,6016,1896,4120
2,X,2781635,chrX:2781635:G:A,rs60075487,A,G,0.114979,0.077130,1.121850,0.964451,...,0.136048,0.153757,0.165084,0.148544,925,313,612,6016,1896,4120
3,X,2781927,chrX:2781927:A:G,rs2306737,A,G,0.053882,0.057451,1.055360,0.942968,...,0.348336,0.436336,0.455169,0.427670,2625,863,1762,6016,1896,4120
4,X,2781986,chrX:2781986:T:C,rs2306736,T,C,0.068425,0.057451,1.070820,0.956783,...,0.233619,0.437334,0.457806,0.427913,2631,868,1763,6016,1896,4120


In [155]:
import warnings
warnings.filterwarnings('ignore')

brain_list = ['Brain_Amygdala','Brain_Anterior_cingulate_cortex_BA24','Brain_Caudate_basal_ganglia','Brain_Cerebellar_Hemisphere','Brain_Cerebellum','Brain_Cortex','Brain_Frontal_Cortex_BA9','Brain_Hippocampus','Brain_Hypothalamus','Brain_Nucleus_accumbens_basal_ganglia','Brain_Putamen_basal_ganglia','Brain_Spinal_cord_cervical_c-1','Brain_Substantia_nigra']

for i in brain_list:
    input = "/data/NDRS_LNG/GTEX_v8/GTEX_v8_chrX_brain/eQTL/" + i + ".v8.EUR.allpairs.chrX.txt"
    eqtl = pd.read_csv(input,sep="\t")
    
    temp = eqtl['variant_id'].str.split("_", expand=True)
    temp.columns = ['chrom','POS','ref','alt','build']
    temp.head()
    
    eqtl_updated = pd.concat([eqtl,temp[['chrom','POS','ref','alt']]], axis=1)
    eqtl_updated['var_id'] = eqtl_updated['chrom'] + ":" + eqtl_updated['POS'] + ":" + eqtl_updated['ref'] + ":" + eqtl_updated['alt']
    eqtl_updated['var_id_alt'] = eqtl_updated['chrom'] + ":" + eqtl_updated['POS'] + ":" + eqtl_updated['alt'] + ":" + eqtl_updated['ref']
    
    # check against xwas summstats SNP ID
    ## eqtl_updated var_id that matches chrom:pos:ref:alt
    temp1 = eqtl_updated[(eqtl_updated.var_id.isin(xwas.ID))]
    temp1['ID'] = temp1['var_id']
    temp1['REF'] = temp1['ref']
    temp1['ALT'] = temp1['alt']
    temp1 = temp1[['phenotype_id', 'variant_id', 'tss_distance', 'ma_samples', 'ma_count',
       'pval_nominal', 'beta', 'varbeta', 'N', 'MAF', 'chrom', 'POS', 'REF',
       'ALT', 'ID']]
    print(temp1.shape)

    ## eqtl_updated var_id_alt that matches chrom:pos:ref:alt
    # for those that matches, will need to flip the beta and MAF values
    temp2 = eqtl_updated[(eqtl_updated.var_id_alt.isin(xwas.ID))]
    print(temp2.shape)
    temp2['beta_corr'] = -1 * temp2['beta']
    temp2['MAF_corr'] = 0.5 - temp2['MAF']
    temp2['ID'] = temp2['var_id_alt']
    temp2['REF'] = temp2['alt']
    temp2['ALT'] = temp2['ref']
    temp2 = temp2[['phenotype_id', 'variant_id', 'tss_distance', 'ma_samples', 'ma_count',
       'pval_nominal', 'beta_corr', 'varbeta', 'N', 'MAF_corr', 'chrom', 'POS', 'REF',
       'ALT', 'ID']]
    temp2 = temp2.rename(columns={'beta_corr':'beta', 'MAF_corr':'MAF'})

    # concat both temp1 and temp2 dataframes - these will only contain variants that are shared between xwas summ stats and eqtl
    eqtl_updated_clean = pd.concat([temp1,temp2], axis = 0)
    print(eqtl_updated_clean.shape)
    
    # get rsid from xwas summ stats
    rsid_map = xwas[['ID','rsID']]
    eqtl_updated_clean_rsid = eqtl_updated_clean.merge(rsid_map, on="ID")
    eqtl_updated_clean_rsid.to_csv("Analysis.COLOC_redo/GTEXv8_eQTL_brains/" + i + ".v8.EUR.allpairs.chrX_harmonized_snpid_shared.txt",sep="\t",index=False,header=True)


(2074203, 15)
(0, 16)
(2074203, 15)
(2086703, 15)
(0, 16)
(2086703, 15)
(2090599, 15)
(0, 16)
(2090599, 15)
(2085779, 15)
(0, 16)
(2085779, 15)
(2087214, 15)
(0, 16)
(2087214, 15)
(2086016, 15)
(0, 16)
(2086016, 15)
(2096348, 15)
(0, 16)
(2096348, 15)
(2052472, 15)
(0, 16)
(2052472, 15)
(2166400, 15)
(0, 16)
(2166400, 15)
(2133849, 15)
(0, 16)
(2133849, 15)
(1997686, 15)
(0, 16)
(1997686, 15)
(2091078, 15)
(0, 16)
(2091078, 15)
(2022392, 15)
(0, 16)
(2022392, 15)


In [156]:
import warnings
warnings.filterwarnings('ignore')

brain_list = ['Brain_Amygdala','Brain_Anterior_cingulate_cortex_BA24','Brain_Caudate_basal_ganglia','Brain_Cerebellar_Hemisphere','Brain_Cerebellum','Brain_Cortex','Brain_Frontal_Cortex_BA9','Brain_Hippocampus','Brain_Hypothalamus','Brain_Nucleus_accumbens_basal_ganglia','Brain_Putamen_basal_ganglia','Brain_Spinal_cord_cervical_c-1','Brain_Substantia_nigra']

for i in brain_list:
    input = "/data/NDRS_LNG/GTEX_v8/GTEX_v8_chrX_brain/eQTL/" + i + ".v8.EUR.signif_pairs.txt"
    eqtl = pd.read_csv(input,sep="\t")
    
    temp = eqtl['variant_id'].str.split("_", expand=True)
    temp.columns = ['chrom','POS','ref','alt','build']
    temp.head()
    
    eqtl_updated = pd.concat([eqtl,temp[['chrom','POS','ref','alt']]], axis=1)
    eqtl_updated['var_id'] = eqtl_updated['chrom'] + ":" + eqtl_updated['POS'] + ":" + eqtl_updated['ref'] + ":" + eqtl_updated['alt']
    eqtl_updated['var_id_alt'] = eqtl_updated['chrom'] + ":" + eqtl_updated['POS'] + ":" + eqtl_updated['alt'] + ":" + eqtl_updated['ref']
    
    # check against xwas summstats SNP ID
    ## eqtl_updated var_id that matches chrom:pos:ref:alt
    temp1 = eqtl_updated[(eqtl_updated.var_id.isin(xwas.ID))]
    temp1['ID'] = temp1['var_id']
    temp1['REF'] = temp1['ref']
    temp1['ALT'] = temp1['alt']
    temp1 = temp1[['phenotype_id', 'variant_id', 'tss_distance', 'ma_samples', 'ma_count',
       'pval_nominal', 'beta', 'varbeta', 'N', 'MAF', 'chrom', 'POS', 'REF',
       'ALT', 'ID']]
    print(temp1.shape)

    ## eqtl_updated var_id_alt that matches chrom:pos:ref:alt
    # for those that matches, will need to flip the beta and MAF values
    temp2 = eqtl_updated[(eqtl_updated.var_id_alt.isin(xwas.ID))]
    print(temp2.shape)
    temp2['beta_corr'] = -1 * temp2['beta']
    temp2['MAF_corr'] = 0.5 - temp2['MAF']
    temp2['ID'] = temp2['var_id_alt']
    temp2['REF'] = temp2['alt']
    temp2['ALT'] = temp2['ref']
    temp2 = temp2[['phenotype_id', 'variant_id', 'tss_distance', 'ma_samples', 'ma_count',
       'pval_nominal', 'beta_corr', 'varbeta', 'N', 'MAF_corr', 'chrom', 'POS', 'REF',
       'ALT', 'ID']]
    temp2 = temp2.rename(columns={'beta_corr':'beta', 'MAF_corr':'MAF'})

    # concat both temp1 and temp2 dataframes - these will only contain variants that are shared between xwas summ stats and eqtl
    eqtl_updated_clean = pd.concat([temp1,temp2], axis = 0)
    print(eqtl_updated_clean.shape)
    
    # get rsid from xwas summ stats
    rsid_map = xwas[['ID','rsID']]
    eqtl_updated_clean_rsid = eqtl_updated_clean.merge(rsid_map, on="ID")
    eqtl_updated_clean_rsid.to_csv("Analysis.COLOC_redo/GTEXv8_eQTL_brains/" + i + ".v8.EUR.signif_pairs.chrX_harmonized_snpid_shared.txt",sep="\t",index=False,header=True)


(5566, 15)
(0, 16)
(5566, 15)
(7962, 15)
(0, 16)
(7962, 15)
(16002, 15)
(0, 16)
(16002, 15)
(19720, 15)
(0, 16)
(19720, 15)
(27471, 15)
(0, 16)
(27471, 15)
(18755, 15)
(0, 16)
(18755, 15)
(17151, 15)
(0, 16)
(17151, 15)
(8407, 15)
(0, 16)
(8407, 15)
(10050, 15)
(0, 16)
(10050, 15)
(16032, 15)
(0, 16)
(16032, 15)
(10713, 15)
(0, 16)
(10713, 15)
(8080, 15)
(0, 16)
(8080, 15)
(3999, 15)
(0, 16)
(3999, 15)


In [157]:
import warnings
warnings.filterwarnings('ignore')

brain_list = ['Brain_Amygdala','Brain_Anterior_cingulate_cortex_BA24','Brain_Caudate_basal_ganglia','Brain_Cerebellar_Hemisphere','Brain_Cerebellum','Brain_Cortex','Brain_Frontal_Cortex_BA9','Brain_Hippocampus','Brain_Hypothalamus','Brain_Nucleus_accumbens_basal_ganglia','Brain_Putamen_basal_ganglia','Brain_Spinal_cord_cervical_c-1','Brain_Substantia_nigra']

for i in brain_list:
    input = "/data/NDRS_LNG/GTEX_v8/GTEX_v8_chrX_brain/eQTL/" + i + ".v8.EUR.egenes.txt"
    eqtl = pd.read_csv(input,sep="\t")
    
    temp = eqtl['variant_id'].str.split("_", expand=True)
    temp.columns = ['chrom','POS','ref','alt','build']
    temp.head()
    
    eqtl_updated = pd.concat([eqtl,temp[['chrom','POS','ref','alt']]], axis=1)
    eqtl_updated['var_id'] = eqtl_updated['chrom'] + ":" + eqtl_updated['POS'] + ":" + eqtl_updated['ref'] + ":" + eqtl_updated['alt']
    eqtl_updated['var_id_alt'] = eqtl_updated['chrom'] + ":" + eqtl_updated['POS'] + ":" + eqtl_updated['alt'] + ":" + eqtl_updated['ref']
    
    # check against xwas summstats SNP ID
    ## eqtl_updated var_id that matches chrom:pos:ref:alt
    temp1 = eqtl_updated[(eqtl_updated.var_id.isin(xwas.ID))]
    temp1['ID'] = temp1['var_id']
    temp1['REF'] = temp1['ref']
    temp1['ALT'] = temp1['alt']
    temp1 = temp1[['phenotype_id', 'variant_id', 'tss_distance', 'ma_samples', 'ma_count',
       'pval_nominal', 'beta', 'varbeta', 'N', 'MAF', 'chrom', 'POS', 'REF',
       'ALT', 'ID']]
    print(temp1.shape)

    ## eqtl_updated var_id_alt that matches chrom:pos:ref:alt
    # for those that matches, will need to flip the beta and MAF values
    temp2 = eqtl_updated[(eqtl_updated.var_id_alt.isin(xwas.ID))]
    print(temp2.shape)
    temp2['beta_corr'] = -1 * temp2['beta']
    temp2['MAF_corr'] = 0.5 - temp2['MAF']
    temp2['ID'] = temp2['var_id_alt']
    temp2['REF'] = temp2['alt']
    temp2['ALT'] = temp2['ref']
    temp2 = temp2[['phenotype_id', 'variant_id', 'tss_distance', 'ma_samples', 'ma_count',
       'pval_nominal', 'beta_corr', 'varbeta', 'N', 'MAF_corr', 'chrom', 'POS', 'REF',
       'ALT', 'ID']]
    temp2 = temp2.rename(columns={'beta_corr':'beta', 'MAF_corr':'MAF'})

    # concat both temp1 and temp2 dataframes - these will only contain variants that are shared between xwas summ stats and eqtl
    eqtl_updated_clean = pd.concat([temp1,temp2], axis = 0)
    print(eqtl_updated_clean.shape)
    
    # get rsid from xwas summ stats
    rsid_map = xwas[['ID','rsID']]
    eqtl_updated_clean_rsid = eqtl_updated_clean.merge(rsid_map, on="ID")
    eqtl_updated_clean_rsid.to_csv("Analysis.COLOC_redo/GTEXv8_eQTL_brains/" + i + ".v8.EUR.egenes.chrX_harmonized_snpid_shared.txt",sep="\t",index=False,header=True)


(675, 15)
(0, 16)
(675, 15)
(688, 15)
(0, 16)
(688, 15)
(679, 15)
(0, 16)
(679, 15)
(687, 15)
(0, 16)
(687, 15)
(693, 15)
(0, 16)
(693, 15)
(686, 15)
(0, 16)
(686, 15)
(683, 15)
(0, 16)
(683, 15)
(680, 15)
(0, 16)
(680, 15)
(704, 15)
(0, 16)
(704, 15)
(690, 15)
(0, 16)
(690, 15)
(635, 15)
(0, 16)
(635, 15)
(698, 15)
(0, 16)
(698, 15)
(652, 15)
(0, 16)
(652, 15)


### run COLOC for each brain tissue and for each subsetted xwas region

#### Download LD binaries for creating LD matrix for analysis using `coloc.susie`

ref link: `https://mrcieu.github.io/ieugwasr/reference/ld_matrix.html`


In [ ]:
%%bash
cd Analysis.COLOC_redo/
mkdir LD
cd LD
wget http://fileserve.mrcieu.ac.uk/ld/1kg.v3.tgz

In [ ]:
%%bash
cd Analysis.COLOC_redo/LD
tar -xvzf 1kg.v3.tgz

#### females-only (unconditioned on APOE4)
##### index_var1 (rs141773145)

In [158]:
%%bash
cd Analysis.COLOC_redo/females_only

module load R/4.3
R --vanilla --no-save

require(data.table)
require(tidyverse)
# install.packages("coloc")
require(coloc)
#devtools::install_github("mrcieu/ieugwasr", force=TRUE)
require(ieugwasr)
#devtools::install_github("explodecomputer/genetics.binaRies")
genetics.binaRies::get_plink_binary()

brain_tissues <- c("Brain_Amygdala","Brain_Anterior_cingulate_cortex_BA24","Brain_Caudate_basal_ganglia","Brain_Cerebellar_Hemisphere","Brain_Cerebellum","Brain_Cortex","Brain_Frontal_Cortex_BA9","Brain_Hippocampus","Brain_Hypothalamus","Brain_Nucleus_accumbens_basal_ganglia","Brain_Putamen_basal_ganglia","Brain_Spinal_cord_cervical_c-1","Brain_Substantia_nigra")

run_coloc <- function(rsid,i){
    # read in files
    tissue <- brain_tissues[i]
    xwas <- fread(paste("XWAS_summ_stats_",rsid,"_flank1Mb_no-maf-filter.txt",sep="")) %>% arrange(POS)
    eqtl <- fread(paste("../GTEXv8_eQTL_brains/",tissue,".v8.EUR.allpairs.chrX_harmonized_snpid_shared.txt",sep="")) %>% arrange(POS)
    dim(eqtl)
    dim(xwas)    
    
    # prep so that only columns that are needed for coloc is present
    eqtl_subset <- subset(eqtl, eqtl$ID %in% xwas$ID) %>%
                   mutate(position = POS,
                          snp = ID) %>%
                   filter(!is.na(beta)) %>%
                   filter(rsID != "") %>%
                   select(rsID, snp, position, beta, varbeta, N, MAF, phenotype_id, pval_nominal) %>%
                   arrange(snp,pval_nominal) %>%
                   group_by(snp) %>%
                   slice(1:1) %>%
                   arrange(position) %>%
                   data.frame()
  
    xwas_subset <- subset(xwas, xwas$ID %in% eqtl_subset$snp) %>%
                   filter(rsID != "") %>%
                   mutate(position = POS,
                          snp = ID, 
                          beta = BETA,
                          varbeta = (BETA_SE**2)) %>%
                   select(rsID, snp, position, beta, varbeta,P)
    
    # check dimensions
    dim(eqtl_subset)
    dim(xwas_subset)
        
    # convert to acceptable data structure for input in coloc
    eqtl_subset_list <- as.list(eqtl_subset)
    eqtl_subset_list$type <- "quant"
    xwas_subset_list <- as.list(xwas_subset)
    xwas_subset_list$type <- "cc"
    xwas_subset_list$N <- 2591 + 4023
    #check_dataset(eqtl_subset_list)
    #check_dataset(xwas_subset_list)
        
    # run coloc.abf
    print(paste("run coloc for: ",tissue,sep=""))
    my.res <- coloc.abf(dataset1=xwas_subset_list,
                    dataset2=eqtl_subset_list)
    print(my.res)
    output_name <- paste("coloc_",rsid,"_",tissue, sep="")

    summary <- my.res$summary %>% data.frame() %>% t() %>% data.frame() %>% mutate(Tissue = tissue, index_variant = rsid)
    results <- my.res$results %>% data.frame() %>% mutate(Tissue = tissue,index_variant = rsid)
    priors <- my.res$priors %>% data.frame() %>% t() %>% data.frame() %>% mutate(Tissue = tissue, index_variant = rsid)
    
    summary
    results
    priors
    
    write.table(summary, paste(output_name,"_summary.txt",sep=""), quote=F, sep="\t", row.names=F, col.names=T)
    write.table(results, paste(output_name,"_results.txt",sep=""), quote=F, sep="\t", row.names=F, col.names=T)
    write.table(priors, paste(output_name,"_priors.txt",sep=""), quote=F, sep="\t", row.names=F, col.names=T)
    print(paste("saving coloc results for: ",tissue,sep=""))
}

for (i in 1:length(brain_tissues)){
    run_coloc("rs141773145", i)
}

[-] Unloading gcc  11.3.0  ... 
[-] Unloading HDF5  1.12.2 
[-] Unloading netcdf  4.9.0 
[-] Unloading openmpi/4.1.3/gcc-11.3.0  ... 
[-] Unloading pandoc  2.18  on cn3180 
[-] Unloading R 4.3.0 
[+] Loading gcc  11.3.0  ... 
[+] Loading HDF5  1.12.2 
[+] Loading netcdf  4.9.0 
[-] Unloading gcc  11.3.0  ... 
[+] Loading gcc  11.3.0  ... 
[+] Loading openmpi/4.1.3/gcc-11.3.0  ... 
[+] Loading pandoc  2.18  on cn3180 
[+] Loading R 4.3.0 



R version 4.3.0 (2023-04-21) -- "Already Tomorrow"
Copyright (C) 2023 The R Foundation for Statistical Computing
Platform: x86_64-pc-linux-gnu (64-bit)

R is free software and comes with ABSOLUTELY NO WARRANTY.
You are welcome to redistribute it under certain conditions.
Type 'license()' or 'licence()' for distribution details.

R is a collaborative project with many contributors.
Type 'contributors()' for more information and
'citation()' on how to cite R or R packages in publications.

Type 'demo()' for some demos, 'help()' for on-line help, or
'help.start()' for an HTML browser interface to help.
Type 'q()' to quit R.

> 
> require(data.table)


Loading required package: data.table


> require(tidyverse)


Loading required package: tidyverse
── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::between()     masks data.table::between()
✖ dplyr::filter()      masks stats::filter()
✖ dplyr::first()       masks data.table::first()
✖ lubridate::hour()    masks data.table::hour()
✖ lubridate::isoweek() masks data.table::isoweek()
✖ dplyr::lag()         masks stats::lag()
✖ dplyr::last()        masks data.table::last()
✖ lubridate::mday()    masks data.table::mday()
✖ lubridate::minute()  masks data.table::minute()
✖ lubridate::month()   masks data.table::month()
✖ lubridate::quarter() masks data.table::quarter()
✖ lubridate::second()  masks data.table::second()
✖ purrr::transpose()   masks data.tab

...

[1] "saving coloc results for: Brain_Spinal_cord_cervical_c-1"
[1] "run coloc for: Brain_Substantia_nigra"
PP.H0.abf PP.H1.abf PP.H2.abf PP.H3.abf PP.H4.abf 
   0.6540    0.1000    0.2010    0.0308    0.0140 
[1] "PP abf for shared variant: 1.4%"
   p1    p2   p12 
1e-04 1e-04 1e-05 
        H0     H1     H2        H3      H4
 0.5385197 0.2006 0.2006 0.0402203 0.02006
       nsnps           H0           H1           H2           H3           H4 
2.006000e+03 6.537681e-01 1.000758e-01 2.013055e-01 3.080086e-02 1.404979e-02 


Coloc analysis of trait 1, trait 2

SNP Priors

Hypothesis Priors

Posterior


[1] "saving coloc results for: Brain_Substantia_nigra"


There were 29 warnings (use warnings() to see them)


> 


##### index_var2 (rs12860838)

In [159]:
%%bash
cd Analysis.COLOC_redo/females_only

module load R/4.3
R --vanilla --no-save

require(data.table)
require(tidyverse)
# install.packages("coloc")
require(coloc)
#devtools::install_github("mrcieu/ieugwasr", force=TRUE)
require(ieugwasr)
#devtools::install_github("explodecomputer/genetics.binaRies")
genetics.binaRies::get_plink_binary()

brain_tissues <- c("Brain_Amygdala","Brain_Anterior_cingulate_cortex_BA24","Brain_Caudate_basal_ganglia","Brain_Cerebellar_Hemisphere","Brain_Cerebellum","Brain_Cortex","Brain_Frontal_Cortex_BA9","Brain_Hippocampus","Brain_Hypothalamus","Brain_Nucleus_accumbens_basal_ganglia","Brain_Putamen_basal_ganglia","Brain_Spinal_cord_cervical_c-1","Brain_Substantia_nigra")

run_coloc <- function(rsid,i){
    # read in files
    tissue <- brain_tissues[i]
    xwas <- fread(paste("XWAS_summ_stats_",rsid,"_flank1Mb_no-maf-filter.txt",sep="")) %>% arrange(POS)
    eqtl <- fread(paste("../GTEXv8_eQTL_brains/",tissue,".v8.EUR.allpairs.chrX_harmonized_snpid_shared.txt",sep="")) %>% arrange(POS)
    dim(eqtl)
    dim(xwas)    
    
    # prep so that only columns that are needed for coloc is present
    eqtl_subset <- subset(eqtl, eqtl$ID %in% xwas$ID) %>%
                   mutate(position = POS,
                          snp = ID) %>%
                   filter(!is.na(beta)) %>%
                   filter(rsID != "") %>%
                   select(rsID, snp, position, beta, varbeta, N, MAF, phenotype_id, pval_nominal) %>%
                   arrange(snp,pval_nominal) %>%
                   group_by(snp) %>%
                   slice(1:1) %>%
                   arrange(position) %>%
                   data.frame()
  
    xwas_subset <- subset(xwas, xwas$ID %in% eqtl_subset$snp) %>%
                   filter(rsID != "") %>%
                   mutate(position = POS,
                          snp = ID, 
                          beta = BETA,
                          varbeta = (BETA_SE**2)) %>%
                   select(rsID, snp, position, beta, varbeta,P)
    
    # check dimensions
    dim(eqtl_subset)
    dim(xwas_subset)
        
    # convert to acceptable data structure for input in coloc
    eqtl_subset_list <- as.list(eqtl_subset)
    eqtl_subset_list$type <- "quant"
    xwas_subset_list <- as.list(xwas_subset)
    xwas_subset_list$type <- "cc"
    xwas_subset_list$N <- 2591 + 4023
    #check_dataset(eqtl_subset_list)
    #check_dataset(xwas_subset_list)
        
    # run coloc.abf
    print(paste("run coloc for: ",tissue,sep=""))
    my.res <- coloc.abf(dataset1=xwas_subset_list,
                    dataset2=eqtl_subset_list)
    print(my.res)
    output_name <- paste("coloc_",rsid,"_",tissue, sep="")

    summary <- my.res$summary %>% data.frame() %>% t() %>% data.frame() %>% mutate(Tissue = tissue, index_variant = rsid)
    results <- my.res$results %>% data.frame() %>% mutate(Tissue = tissue,index_variant = rsid)
    priors <- my.res$priors %>% data.frame() %>% t() %>% data.frame() %>% mutate(Tissue = tissue, index_variant = rsid)
    
    write.table(summary, paste(output_name,"_summary.txt",sep=""), quote=F, sep="\t", row.names=F, col.names=T)
    write.table(results, paste(output_name,"_results.txt",sep=""), quote=F, sep="\t", row.names=F, col.names=T)
    write.table(priors, paste(output_name,"_priors.txt",sep=""), quote=F, sep="\t", row.names=F, col.names=T)
    print(paste("saving coloc results for: ",tissue,sep=""))
}

for (i in 1:length(brain_tissues)){
    run_coloc("rs12860838", i)
}


[-] Unloading gcc  11.3.0  ... 
[-] Unloading HDF5  1.12.2 
[-] Unloading netcdf  4.9.0 
[-] Unloading openmpi/4.1.3/gcc-11.3.0  ... 
[-] Unloading pandoc  2.18  on cn3180 
[-] Unloading R 4.3.0 
[+] Loading gcc  11.3.0  ... 
[+] Loading HDF5  1.12.2 
[+] Loading netcdf  4.9.0 
[-] Unloading gcc  11.3.0  ... 
[+] Loading gcc  11.3.0  ... 
[+] Loading openmpi/4.1.3/gcc-11.3.0  ... 
[+] Loading pandoc  2.18  on cn3180 
[+] Loading R 4.3.0 



R version 4.3.0 (2023-04-21) -- "Already Tomorrow"
Copyright (C) 2023 The R Foundation for Statistical Computing
Platform: x86_64-pc-linux-gnu (64-bit)

R is free software and comes with ABSOLUTELY NO WARRANTY.
You are welcome to redistribute it under certain conditions.
Type 'license()' or 'licence()' for distribution details.

R is a collaborative project with many contributors.
Type 'contributors()' for more information and
'citation()' on how to cite R or R packages in publications.

Type 'demo()' for some demos, 'help()' for on-line help, or
'help.start()' for an HTML browser interface to help.
Type 'q()' to quit R.

> 
> require(data.table)


Loading required package: data.table


> require(tidyverse)


Loading required package: tidyverse
── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::between()     masks data.table::between()
✖ dplyr::filter()      masks stats::filter()
✖ dplyr::first()       masks data.table::first()
✖ lubridate::hour()    masks data.table::hour()
✖ lubridate::isoweek() masks data.table::isoweek()
✖ dplyr::lag()         masks stats::lag()
✖ dplyr::last()        masks data.table::last()
✖ lubridate::mday()    masks data.table::mday()
✖ lubridate::minute()  masks data.table::minute()
✖ lubridate::month()   masks data.table::month()
✖ lubridate::quarter() masks data.table::quarter()
✖ lubridate::second()  masks data.table::second()
✖ purrr::transpose()   masks data.tab

...

Coloc analysis of trait 1, trait 2

SNP Priors

Hypothesis Priors

Posterior


   p1    p2   p12 
1e-04 1e-04 1e-05 
         H0     H1     H2        H3      H4
 -0.2573618 0.4862 0.4862 0.2363418 0.04862
       nsnps           H0           H1           H2           H3           H4 
4.862000e+03 2.342308e-01 3.410173e-01 1.594840e-01 2.321601e-01 3.310783e-02 
[1] "saving coloc results for: Brain_Spinal_cord_cervical_c-1"
[1] "run coloc for: Brain_Substantia_nigra"
PP.H0.abf PP.H1.abf PP.H2.abf PP.H3.abf PP.H4.abf 
    0.253     0.362     0.135     0.193     0.057 
[1] "PP abf for shared variant: 5.7%"


Coloc analysis of trait 1, trait 2

SNP Priors

Hypothesis Priors

Posterior


   p1    p2   p12 
1e-04 1e-04 1e-05 
         H0     H1     H2        H3      H4
 -0.1139816 0.4388 0.4388 0.1925016 0.04388
       nsnps           H0           H1           H2           H3           H4 
4.388000e+03 2.532806e-01 3.617263e-01 1.351027e-01 1.928918e-01 5.699862e-02 
[1] "saving coloc results for: Brain_Substantia_nigra"
> 


There were 38 warnings (use warnings() to see them)


#### females-only (conditioned on APOE4)
##### index_var1 (rs141773145)

In [160]:
%%bash
cd Analysis.COLOC_redo/females_only_cond_ApoE4/

module load R/4.3
R --vanilla --no-save

require(data.table)
require(tidyverse)
# install.packages("coloc")
require(coloc)
#devtools::install_github("mrcieu/ieugwasr", force=TRUE)
require(ieugwasr)
#devtools::install_github("explodecomputer/genetics.binaRies")
genetics.binaRies::get_plink_binary()

brain_tissues <- c("Brain_Amygdala","Brain_Anterior_cingulate_cortex_BA24","Brain_Caudate_basal_ganglia","Brain_Cerebellar_Hemisphere","Brain_Cerebellum","Brain_Cortex","Brain_Frontal_Cortex_BA9","Brain_Hippocampus","Brain_Hypothalamus","Brain_Nucleus_accumbens_basal_ganglia","Brain_Putamen_basal_ganglia","Brain_Spinal_cord_cervical_c-1","Brain_Substantia_nigra")

run_coloc <- function(rsid,i){
    # read in files
    tissue <- brain_tissues[i]
    xwas <- fread(paste("XWAS_summ_stats_",rsid,"_flank1Mb_no-maf-filter.txt",sep="")) %>% arrange(POS)
    eqtl <- fread(paste("../GTEXv8_eQTL_brains/",tissue,".v8.EUR.allpairs.chrX_harmonized_snpid_shared.txt",sep="")) %>% arrange(POS)
    dim(eqtl)
    dim(xwas)    
    
    # prep so that only columns that are needed for coloc is present
    eqtl_subset <- subset(eqtl, eqtl$ID %in% xwas$ID) %>%
                   mutate(position = POS,
                          snp = ID) %>%
                   filter(!is.na(beta)) %>%
                   filter(rsID != "") %>%
                   select(rsID, snp, position, beta, varbeta, N, MAF, phenotype_id, pval_nominal) %>%
                   arrange(snp,pval_nominal) %>%
                   group_by(snp) %>%
                   slice(1:1) %>%
                   arrange(position) %>%
                   data.frame()
  
    xwas_subset <- subset(xwas, xwas$ID %in% eqtl_subset$snp) %>%
                   filter(rsID != "") %>%
                   mutate(position = POS,
                          snp = ID, 
                          beta = BETA,
                          varbeta = (BETA_SE**2)) %>%
                   select(rsID, snp, position, beta, varbeta,P)
    
    # check dimensions
    dim(eqtl_subset)
    dim(xwas_subset)
        
    # convert to acceptable data structure for input in coloc
    eqtl_subset_list <- as.list(eqtl_subset)
    eqtl_subset_list$type <- "quant"
    xwas_subset_list <- as.list(xwas_subset)
    xwas_subset_list$type <- "cc"
    xwas_subset_list$N <- 2591 + 4023
    #check_dataset(eqtl_subset_list)
    #check_dataset(xwas_subset_list)
        
    # run coloc.abf
    print(paste("run coloc for: ",tissue,sep=""))
    my.res <- coloc.abf(dataset1=xwas_subset_list,
                    dataset2=eqtl_subset_list)
    print(my.res)
    output_name <- paste("coloc_",rsid,"_",tissue, sep="")

    summary <- my.res$summary %>% data.frame() %>% t() %>% data.frame() %>% mutate(Tissue = tissue, index_variant = rsid)
    results <- my.res$results %>% data.frame() %>% mutate(Tissue = tissue,index_variant = rsid)
    priors <- my.res$priors %>% data.frame() %>% t() %>% data.frame() %>% mutate(Tissue = tissue, index_variant = rsid)
    
    write.table(summary, paste(output_name,"_summary.txt",sep=""), quote=F, sep="\t", row.names=F, col.names=T)
    write.table(results, paste(output_name,"_results.txt",sep=""), quote=F, sep="\t", row.names=F, col.names=T)
    write.table(priors, paste(output_name,"_priors.txt",sep=""), quote=F, sep="\t", row.names=F, col.names=T)
    print(paste("saving coloc results for: ",tissue,sep=""))
}

for (i in 1:length(brain_tissues)){
    run_coloc("rs141773145", i)
}

[-] Unloading gcc  11.3.0  ... 
[-] Unloading HDF5  1.12.2 
[-] Unloading netcdf  4.9.0 
[-] Unloading openmpi/4.1.3/gcc-11.3.0  ... 
[-] Unloading pandoc  2.18  on cn3180 
[-] Unloading R 4.3.0 
[+] Loading gcc  11.3.0  ... 
[+] Loading HDF5  1.12.2 
[+] Loading netcdf  4.9.0 
[-] Unloading gcc  11.3.0  ... 
[+] Loading gcc  11.3.0  ... 
[+] Loading openmpi/4.1.3/gcc-11.3.0  ... 
[+] Loading pandoc  2.18  on cn3180 
[+] Loading R 4.3.0 



R version 4.3.0 (2023-04-21) -- "Already Tomorrow"
Copyright (C) 2023 The R Foundation for Statistical Computing
Platform: x86_64-pc-linux-gnu (64-bit)

R is free software and comes with ABSOLUTELY NO WARRANTY.
You are welcome to redistribute it under certain conditions.
Type 'license()' or 'licence()' for distribution details.

R is a collaborative project with many contributors.
Type 'contributors()' for more information and
'citation()' on how to cite R or R packages in publications.

Type 'demo()' for some demos, 'help()' for on-line help, or
'help.start()' for an HTML browser interface to help.
Type 'q()' to quit R.

> 
> require(data.table)


Loading required package: data.table


> require(tidyverse)


Loading required package: tidyverse
── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::between()     masks data.table::between()
✖ dplyr::filter()      masks stats::filter()
✖ dplyr::first()       masks data.table::first()
✖ lubridate::hour()    masks data.table::hour()
✖ lubridate::isoweek() masks data.table::isoweek()
✖ dplyr::lag()         masks stats::lag()
✖ dplyr::last()        masks data.table::last()
✖ lubridate::mday()    masks data.table::mday()
✖ lubridate::minute()  masks data.table::minute()
✖ lubridate::month()   masks data.table::month()
✖ lubridate::quarter() masks data.table::quarter()
✖ lubridate::second()  masks data.table::second()
✖ purrr::transpose()   masks data.tab

...

Coloc analysis of trait 1, trait 2

SNP Priors

Hypothesis Priors

Posterior


   p1    p2   p12 
1e-04 1e-04 1e-05 
        H0     H1     H2         H3      H4
 0.5357673 0.2017 0.2017 0.04066272 0.02017
       nsnps           H0           H1           H2           H3           H4 
2.017000e+03 1.449731e-01 2.442240e-02 6.869839e-01 1.157025e-01 2.791806e-02 
[1] "saving coloc results for: Brain_Spinal_cord_cervical_c-1"
[1] "run coloc for: Brain_Substantia_nigra"
PP.H0.abf PP.H1.abf PP.H2.abf PP.H3.abf PP.H4.abf 
   0.6450    0.1080    0.1980    0.0332    0.0157 
[1] "PP abf for shared variant: 1.57%"


Coloc analysis of trait 1, trait 2

SNP Priors

Hypothesis Priors

Posterior


   p1    p2   p12 
1e-04 1e-04 1e-05 
        H0     H1     H2        H3      H4
 0.5387698 0.2005 0.2005 0.0401802 0.02005
       nsnps           H0           H1           H2           H3           H4 
2.005000e+03 6.447120e-01 1.079086e-01 1.984477e-01 3.319942e-02 1.573231e-02 
[1] "saving coloc results for: Brain_Substantia_nigra"
> 


There were 29 warnings (use warnings() to see them)


##### index_var2 (rs6648060)

In [161]:
%%bash
cd Analysis.COLOC_redo/females_only_cond_ApoE4/

module load R/4.3
R --vanilla --no-save

require(data.table)
require(tidyverse)
# install.packages("coloc")
require(coloc)
#devtools::install_github("mrcieu/ieugwasr", force=TRUE)
require(ieugwasr)
#devtools::install_github("explodecomputer/genetics.binaRies")
genetics.binaRies::get_plink_binary()

brain_tissues <- c("Brain_Amygdala","Brain_Anterior_cingulate_cortex_BA24","Brain_Caudate_basal_ganglia","Brain_Cerebellar_Hemisphere","Brain_Cerebellum","Brain_Cortex","Brain_Frontal_Cortex_BA9","Brain_Hippocampus","Brain_Hypothalamus","Brain_Nucleus_accumbens_basal_ganglia","Brain_Putamen_basal_ganglia","Brain_Spinal_cord_cervical_c-1","Brain_Substantia_nigra")

run_coloc <- function(rsid,i){
    # read in files
    tissue <- brain_tissues[i]
    xwas <- fread(paste("XWAS_summ_stats_",rsid,"_flank1Mb_no-maf-filter.txt",sep="")) %>% arrange(POS)
    eqtl <- fread(paste("../GTEXv8_eQTL_brains/",tissue,".v8.EUR.allpairs.chrX_harmonized_snpid_shared.txt",sep="")) %>% arrange(POS)
    dim(eqtl)
    dim(xwas)    
    
    # prep so that only columns that are needed for coloc is present
    eqtl_subset <- subset(eqtl, eqtl$ID %in% xwas$ID) %>%
                   mutate(position = POS,
                          snp = ID) %>%
                   filter(!is.na(beta)) %>%
                   filter(rsID != "") %>%
                   select(rsID, snp, position, beta, varbeta, N, MAF, phenotype_id, pval_nominal) %>%
                   arrange(snp,pval_nominal) %>%
                   group_by(snp) %>%
                   slice(1:1) %>%
                   arrange(position) %>%
                   data.frame()
  
    xwas_subset <- subset(xwas, xwas$ID %in% eqtl_subset$snp) %>%
                   filter(rsID != "") %>%
                   mutate(position = POS,
                          snp = ID, 
                          beta = BETA,
                          varbeta = (BETA_SE**2)) %>%
                   select(rsID, snp, position, beta, varbeta,P)
    
    # check dimensions
    dim(eqtl_subset)
    dim(xwas_subset)
        
    # convert to acceptable data structure for input in coloc
    eqtl_subset_list <- as.list(eqtl_subset)
    eqtl_subset_list$type <- "quant"
    xwas_subset_list <- as.list(xwas_subset)
    xwas_subset_list$type <- "cc"
    xwas_subset_list$N <- 2591 + 4023
    #check_dataset(eqtl_subset_list)
    #check_dataset(xwas_subset_list)
        
    # run coloc.abf
    print(paste("run coloc for: ",tissue,sep=""))
    my.res <- coloc.abf(dataset1=xwas_subset_list,
                    dataset2=eqtl_subset_list)
    print(my.res)
    output_name <- paste("coloc_",rsid,"_",tissue, sep="")

    summary <- my.res$summary %>% data.frame() %>% t() %>% data.frame() %>% mutate(Tissue = tissue, index_variant = rsid)
    results <- my.res$results %>% data.frame() %>% mutate(Tissue = tissue,index_variant = rsid)
    priors <- my.res$priors %>% data.frame() %>% t() %>% data.frame() %>% mutate(Tissue = tissue, index_variant = rsid)
    
    write.table(summary, paste(output_name,"_summary.txt",sep=""), quote=F, sep="\t", row.names=F, col.names=T)
    write.table(results, paste(output_name,"_results.txt",sep=""), quote=F, sep="\t", row.names=F, col.names=T)
    write.table(priors, paste(output_name,"_priors.txt",sep=""), quote=F, sep="\t", row.names=F, col.names=T)
    print(paste("saving coloc results for: ",tissue,sep=""))
}

for (i in 1:length(brain_tissues)){
    run_coloc("rs6648060", i)
}

[-] Unloading gcc  11.3.0  ... 
[-] Unloading HDF5  1.12.2 
[-] Unloading netcdf  4.9.0 
[-] Unloading openmpi/4.1.3/gcc-11.3.0  ... 
[-] Unloading pandoc  2.18  on cn3180 
[-] Unloading R 4.3.0 
[+] Loading gcc  11.3.0  ... 
[+] Loading HDF5  1.12.2 
[+] Loading netcdf  4.9.0 
[-] Unloading gcc  11.3.0  ... 
[+] Loading gcc  11.3.0  ... 
[+] Loading openmpi/4.1.3/gcc-11.3.0  ... 
[+] Loading pandoc  2.18  on cn3180 
[+] Loading R 4.3.0 



R version 4.3.0 (2023-04-21) -- "Already Tomorrow"
Copyright (C) 2023 The R Foundation for Statistical Computing
Platform: x86_64-pc-linux-gnu (64-bit)

R is free software and comes with ABSOLUTELY NO WARRANTY.
You are welcome to redistribute it under certain conditions.
Type 'license()' or 'licence()' for distribution details.

R is a collaborative project with many contributors.
Type 'contributors()' for more information and
'citation()' on how to cite R or R packages in publications.

Type 'demo()' for some demos, 'help()' for on-line help, or
'help.start()' for an HTML browser interface to help.
Type 'q()' to quit R.

> 
> require(data.table)


Loading required package: data.table


> require(tidyverse)


Loading required package: tidyverse
── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::between()     masks data.table::between()
✖ dplyr::filter()      masks stats::filter()
✖ dplyr::first()       masks data.table::first()
✖ lubridate::hour()    masks data.table::hour()
✖ lubridate::isoweek() masks data.table::isoweek()
✖ dplyr::lag()         masks stats::lag()
✖ dplyr::last()        masks data.table::last()
✖ lubridate::mday()    masks data.table::mday()
✖ lubridate::minute()  masks data.table::minute()
✖ lubridate::month()   masks data.table::month()
✖ lubridate::quarter() masks data.table::quarter()
✖ lubridate::second()  masks data.table::second()
✖ purrr::transpose()   masks data.tab

...

Coloc analysis of trait 1, trait 2

SNP Priors

Hypothesis Priors

Posterior


   p1    p2   p12 
1e-04 1e-04 1e-05 
         H0     H1     H2        H3      H4
 -0.0745577 0.4255 0.4255 0.1810077 0.04255
       nsnps           H0           H1           H2           H3           H4 
4.255000e+03 1.474501e-02 2.342721e-02 3.619185e-01 5.749994e-01 2.490988e-02 
[1] "saving coloc results for: Brain_Spinal_cord_cervical_c-1"
[1] "run coloc for: Brain_Substantia_nigra"
PP.H0.abf PP.H1.abf PP.H2.abf PP.H3.abf PP.H4.abf 
   0.1660    0.2640    0.2020    0.3210    0.0482 
[1] "PP abf for shared variant: 4.82%"


Coloc analysis of trait 1, trait 2

SNP Priors

Hypothesis Priors

Posterior


   p1    p2   p12 
1e-04 1e-04 1e-05 
          H0     H1     H2        H3      H4
 -0.07278752 0.4249 0.4249 0.1804975 0.04249
       nsnps           H0           H1           H2           H3           H4 
4.249000e+03 1.659074e-01 2.635033e-01 2.018544e-01 3.205481e-01 4.818679e-02 
[1] "saving coloc results for: Brain_Substantia_nigra"
> 


There were 29 warnings (use warnings() to see them)


##### index_var3 (rs141193614)

In [162]:
%%bash
cd Analysis.COLOC_redo/females_only_cond_ApoE4/

module load R/4.3
R --vanilla --no-save

require(data.table)
require(tidyverse)
# install.packages("coloc")
require(coloc)
#devtools::install_github("mrcieu/ieugwasr", force=TRUE)
require(ieugwasr)
#devtools::install_github("explodecomputer/genetics.binaRies")
genetics.binaRies::get_plink_binary()

brain_tissues <- c("Brain_Amygdala","Brain_Anterior_cingulate_cortex_BA24","Brain_Caudate_basal_ganglia","Brain_Cerebellar_Hemisphere","Brain_Cerebellum","Brain_Cortex","Brain_Frontal_Cortex_BA9","Brain_Hippocampus","Brain_Hypothalamus","Brain_Nucleus_accumbens_basal_ganglia","Brain_Putamen_basal_ganglia","Brain_Spinal_cord_cervical_c-1","Brain_Substantia_nigra")

run_coloc <- function(rsid,i){
    # read in files
    tissue <- brain_tissues[i]
    xwas <- fread(paste("XWAS_summ_stats_",rsid,"_flank1Mb_no-maf-filter.txt",sep="")) %>% arrange(POS)
    eqtl <- fread(paste("../GTEXv8_eQTL_brains/",tissue,".v8.EUR.allpairs.chrX_harmonized_snpid_shared.txt",sep="")) %>% arrange(POS)
    dim(eqtl)
    dim(xwas)    
    
    # prep so that only columns that are needed for coloc is present
    eqtl_subset <- subset(eqtl, eqtl$ID %in% xwas$ID) %>%
                   mutate(position = POS,
                          snp = ID) %>%
                   filter(!is.na(beta)) %>%
                   filter(rsID != "") %>%
                   select(rsID, snp, position, beta, varbeta, N, MAF, phenotype_id, pval_nominal) %>%
                   arrange(snp,pval_nominal) %>%
                   group_by(snp) %>%
                   slice(1:1) %>%
                   arrange(position) %>%
                   data.frame()
  
    xwas_subset <- subset(xwas, xwas$ID %in% eqtl_subset$snp) %>%
                   filter(rsID != "") %>%
                   mutate(position = POS,
                          snp = ID, 
                          beta = BETA,
                          varbeta = (BETA_SE**2)) %>%
                   select(rsID, snp, position, beta, varbeta,P)
    
    # check dimensions
    dim(eqtl_subset)
    dim(xwas_subset)
        
    # convert to acceptable data structure for input in coloc
    eqtl_subset_list <- as.list(eqtl_subset)
    eqtl_subset_list$type <- "quant"
    xwas_subset_list <- as.list(xwas_subset)
    xwas_subset_list$type <- "cc"
    xwas_subset_list$N <- 2591 + 4023
    #check_dataset(eqtl_subset_list)
    #check_dataset(xwas_subset_list)
        
    # run coloc.abf
    print(paste("run coloc for: ",tissue,sep=""))
    my.res <- coloc.abf(dataset1=xwas_subset_list,
                    dataset2=eqtl_subset_list)
    print(my.res)
    output_name <- paste("coloc_",rsid,"_",tissue, sep="")

    summary <- my.res$summary %>% data.frame() %>% t() %>% data.frame() %>% mutate(Tissue = tissue, index_variant = rsid)
    results <- my.res$results %>% data.frame() %>% mutate(Tissue = tissue,index_variant = rsid)
    priors <- my.res$priors %>% data.frame() %>% t() %>% data.frame() %>% mutate(Tissue = tissue, index_variant = rsid)
    
    write.table(summary, paste(output_name,"_summary.txt",sep=""), quote=F, sep="\t", row.names=F, col.names=T)
    write.table(results, paste(output_name,"_results.txt",sep=""), quote=F, sep="\t", row.names=F, col.names=T)
    write.table(priors, paste(output_name,"_priors.txt",sep=""), quote=F, sep="\t", row.names=F, col.names=T)
    print(paste("saving coloc results for: ",tissue,sep=""))
}

for (i in 1:length(brain_tissues)){
    run_coloc("rs141193614", i)
}

[-] Unloading gcc  11.3.0  ... 
[-] Unloading HDF5  1.12.2 
[-] Unloading netcdf  4.9.0 
[-] Unloading openmpi/4.1.3/gcc-11.3.0  ... 
[-] Unloading pandoc  2.18  on cn3180 
[-] Unloading R 4.3.0 
[+] Loading gcc  11.3.0  ... 
[+] Loading HDF5  1.12.2 
[+] Loading netcdf  4.9.0 
[-] Unloading gcc  11.3.0  ... 
[+] Loading gcc  11.3.0  ... 
[+] Loading openmpi/4.1.3/gcc-11.3.0  ... 
[+] Loading pandoc  2.18  on cn3180 
[+] Loading R 4.3.0 



R version 4.3.0 (2023-04-21) -- "Already Tomorrow"
Copyright (C) 2023 The R Foundation for Statistical Computing
Platform: x86_64-pc-linux-gnu (64-bit)

R is free software and comes with ABSOLUTELY NO WARRANTY.
You are welcome to redistribute it under certain conditions.
Type 'license()' or 'licence()' for distribution details.

R is a collaborative project with many contributors.
Type 'contributors()' for more information and
'citation()' on how to cite R or R packages in publications.

Type 'demo()' for some demos, 'help()' for on-line help, or
'help.start()' for an HTML browser interface to help.
Type 'q()' to quit R.

> 
> require(data.table)


Loading required package: data.table


> require(tidyverse)


Loading required package: tidyverse
── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::between()     masks data.table::between()
✖ dplyr::filter()      masks stats::filter()
✖ dplyr::first()       masks data.table::first()
✖ lubridate::hour()    masks data.table::hour()
✖ lubridate::isoweek() masks data.table::isoweek()
✖ dplyr::lag()         masks stats::lag()
✖ dplyr::last()        masks data.table::last()
✖ lubridate::mday()    masks data.table::mday()
✖ lubridate::minute()  masks data.table::minute()
✖ lubridate::month()   masks data.table::month()
✖ lubridate::quarter() masks data.table::quarter()
✖ lubridate::second()  masks data.table::second()
✖ purrr::transpose()   masks data.tab

...

Coloc analysis of trait 1, trait 2

SNP Priors

Hypothesis Priors

Posterior


   p1    p2   p12 
1e-04 1e-04 1e-05 
        H0     H1     H2         H3      H4
 0.6914074 0.1379 0.1379 0.01900262 0.01379
       nsnps           H0           H1           H2           H3           H4 
1.379000e+03 8.039040e-01 7.466283e-02 1.044755e-01 9.695929e-03 7.261761e-03 
[1] "saving coloc results for: Brain_Spinal_cord_cervical_c-1"
[1] "run coloc for: Brain_Substantia_nigra"
PP.H0.abf PP.H1.abf PP.H2.abf PP.H3.abf PP.H4.abf 
 0.956000  0.022200  0.019700  0.000455  0.002100 
[1] "PP abf for shared variant: 0.21%"


Coloc analysis of trait 1, trait 2

SNP Priors

Hypothesis Priors

Posterior


   p1    p2   p12 
1e-04 1e-04 1e-05 
        H0    H1    H2        H3     H4
 0.9533182 0.022 0.022 0.0004818 0.0022
       nsnps           H0           H1           H2           H3           H4 
2.200000e+02 9.555769e-01 2.217815e-02 1.969396e-02 4.549845e-04 2.095985e-03 
[1] "saving coloc results for: Brain_Substantia_nigra"
> 


There were 39 warnings (use warnings() to see them)


##### index_var4 (rs12860838)

In [163]:
%%bash
cd Analysis.COLOC_redo/females_only_cond_ApoE4/

module load R/4.3
R --vanilla --no-save

require(data.table)
require(tidyverse)
# install.packages("coloc")
require(coloc)
#devtools::install_github("mrcieu/ieugwasr", force=TRUE)
require(ieugwasr)
#devtools::install_github("explodecomputer/genetics.binaRies")
genetics.binaRies::get_plink_binary()

brain_tissues <- c("Brain_Amygdala","Brain_Anterior_cingulate_cortex_BA24","Brain_Caudate_basal_ganglia","Brain_Cerebellar_Hemisphere","Brain_Cerebellum","Brain_Cortex","Brain_Frontal_Cortex_BA9","Brain_Hippocampus","Brain_Hypothalamus","Brain_Nucleus_accumbens_basal_ganglia","Brain_Putamen_basal_ganglia","Brain_Spinal_cord_cervical_c-1","Brain_Substantia_nigra")

run_coloc <- function(rsid,i){
    # read in files
    tissue <- brain_tissues[i]
    xwas <- fread(paste("XWAS_summ_stats_",rsid,"_flank1Mb_no-maf-filter.txt",sep="")) %>% arrange(POS)
    eqtl <- fread(paste("../GTEXv8_eQTL_brains/",tissue,".v8.EUR.allpairs.chrX_harmonized_snpid_shared.txt",sep="")) %>% arrange(POS)
    dim(eqtl)
    dim(xwas)    
    
    # prep so that only columns that are needed for coloc is present
    eqtl_subset <- subset(eqtl, eqtl$ID %in% xwas$ID) %>%
                   mutate(position = POS,
                          snp = ID) %>%
                   filter(!is.na(beta)) %>%
                   filter(rsID != "") %>%
                   select(rsID, snp, position, beta, varbeta, N, MAF, phenotype_id, pval_nominal) %>%
                   arrange(snp,pval_nominal) %>%
                   group_by(snp) %>%
                   slice(1:1) %>%
                   arrange(position) %>%
                   data.frame()
  
    xwas_subset <- subset(xwas, xwas$ID %in% eqtl_subset$snp) %>%
                   filter(rsID != "") %>%
                   mutate(position = POS,
                          snp = ID, 
                          beta = BETA,
                          varbeta = (BETA_SE**2)) %>%
                   select(rsID, snp, position, beta, varbeta,P)
    
    # check dimensions
    dim(eqtl_subset)
    dim(xwas_subset)
        
    # convert to acceptable data structure for input in coloc
    eqtl_subset_list <- as.list(eqtl_subset)
    eqtl_subset_list$type <- "quant"
    xwas_subset_list <- as.list(xwas_subset)
    xwas_subset_list$type <- "cc"
    xwas_subset_list$N <- 2591 + 4023
    #check_dataset(eqtl_subset_list)
    #check_dataset(xwas_subset_list)
        
    # run coloc.abf
    print(paste("run coloc for: ",tissue,sep=""))
    my.res <- coloc.abf(dataset1=xwas_subset_list,
                    dataset2=eqtl_subset_list)
    print(my.res)
    output_name <- paste("coloc_",rsid,"_",tissue, sep="")

    summary <- my.res$summary %>% data.frame() %>% t() %>% data.frame() %>% mutate(Tissue = tissue, index_variant = rsid)
    results <- my.res$results %>% data.frame() %>% mutate(Tissue = tissue,index_variant = rsid)
    priors <- my.res$priors %>% data.frame() %>% t() %>% data.frame() %>% mutate(Tissue = tissue, index_variant = rsid)
    
    write.table(summary, paste(output_name,"_summary.txt",sep=""), quote=F, sep="\t", row.names=F, col.names=T)
    write.table(results, paste(output_name,"_results.txt",sep=""), quote=F, sep="\t", row.names=F, col.names=T)
    write.table(priors, paste(output_name,"_priors.txt",sep=""), quote=F, sep="\t", row.names=F, col.names=T)
    print(paste("saving coloc results for: ",tissue,sep=""))
}

for (i in 1:length(brain_tissues)){
    run_coloc("rs12860838", i)
}

[-] Unloading gcc  11.3.0  ... 
[-] Unloading HDF5  1.12.2 
[-] Unloading netcdf  4.9.0 
[-] Unloading openmpi/4.1.3/gcc-11.3.0  ... 
[-] Unloading pandoc  2.18  on cn3180 
[-] Unloading R 4.3.0 
[+] Loading gcc  11.3.0  ... 
[+] Loading HDF5  1.12.2 
[+] Loading netcdf  4.9.0 
[-] Unloading gcc  11.3.0  ... 
[+] Loading gcc  11.3.0  ... 
[+] Loading openmpi/4.1.3/gcc-11.3.0  ... 
[+] Loading pandoc  2.18  on cn3180 
[+] Loading R 4.3.0 



R version 4.3.0 (2023-04-21) -- "Already Tomorrow"
Copyright (C) 2023 The R Foundation for Statistical Computing
Platform: x86_64-pc-linux-gnu (64-bit)

R is free software and comes with ABSOLUTELY NO WARRANTY.
You are welcome to redistribute it under certain conditions.
Type 'license()' or 'licence()' for distribution details.

R is a collaborative project with many contributors.
Type 'contributors()' for more information and
'citation()' on how to cite R or R packages in publications.

Type 'demo()' for some demos, 'help()' for on-line help, or
'help.start()' for an HTML browser interface to help.
Type 'q()' to quit R.

> 
> require(data.table)


Loading required package: data.table


> require(tidyverse)


Loading required package: tidyverse
── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::between()     masks data.table::between()
✖ dplyr::filter()      masks stats::filter()
✖ dplyr::first()       masks data.table::first()
✖ lubridate::hour()    masks data.table::hour()
✖ lubridate::isoweek() masks data.table::isoweek()
✖ dplyr::lag()         masks stats::lag()
✖ dplyr::last()        masks data.table::last()
✖ lubridate::mday()    masks data.table::mday()
✖ lubridate::minute()  masks data.table::minute()
✖ lubridate::month()   masks data.table::month()
✖ lubridate::quarter() masks data.table::quarter()
✖ lubridate::second()  masks data.table::second()
✖ purrr::transpose()   masks data.tab

...

Coloc analysis of trait 1, trait 2

SNP Priors

Hypothesis Priors

Posterior


   p1    p2   p12 
1e-04 1e-04 1e-05 
         H0    H1    H2        H3     H4
 -0.2567474 0.486 0.486 0.2361474 0.0486
       nsnps           H0           H1           H2           H3           H4 
4.860000e+03 1.595135e-01 4.127673e-01 1.085782e-01 2.809256e-01 3.821548e-02 
[1] "saving coloc results for: Brain_Spinal_cord_cervical_c-1"
[1] "run coloc for: Brain_Substantia_nigra"
PP.H0.abf PP.H1.abf PP.H2.abf PP.H3.abf PP.H4.abf 
   0.1710    0.4370    0.0911    0.2330    0.0676 
[1] "PP abf for shared variant: 6.76%"


Coloc analysis of trait 1, trait 2

SNP Priors

Hypothesis Priors

Posterior


   p1    p2   p12 
1e-04 1e-04 1e-05 
         H0     H1     H2        H3      H4
 -0.1133861 0.4386 0.4386 0.1923261 0.04386
       nsnps           H0           H1           H2           H3           H4 
4.386000e+03 1.707745e-01 4.374195e-01 9.105843e-02 2.331683e-01 6.757928e-02 
[1] "saving coloc results for: Brain_Substantia_nigra"
> 


There were 38 warnings (use warnings() to see them)


#### merge coloc results and make posterior probability table

In [164]:
%%bash
cd Analysis.COLOC_redo/

head -n 1 females_only/coloc_rs141773145_Brain_Amygdala_summary.txt > females_only/coloc_posteriors_compiled_allIndexVars_allBrainTissues.txt
cat females_only/coloc_rs141773145_Brain_*_summary.txt females_only/coloc_rs12860838_Brain_*_summary.txt | grep -v "Tissue" >> females_only/coloc_posteriors_compiled_allIndexVars_allBrainTissues.txt


head -n 1 females_only_cond_ApoE4/coloc_rs141773145_Brain_Amygdala_summary.txt > females_only_cond_ApoE4/coloc_posteriors_compiled_allIndexVars_allBrainTissues.txt
cat females_only_cond_ApoE4/coloc_rs141773145_Brain_*_summary.txt females_only_cond_ApoE4/coloc_rs6648060_Brain_*_summary.txt females_only_cond_ApoE4/coloc_rs141193614_Brain_*_summary.txt females_only_cond_ApoE4/coloc_rs12860838_Brain_*_summary.txt | grep -v "Tissue" >> females_only_cond_ApoE4/coloc_posteriors_compiled_allIndexVars_allBrainTissues.txt


In [165]:
import pandas as pd
compiled_post = pd.read_csv("Analysis.COLOC_redo/females_only/coloc_posteriors_compiled_allIndexVars_allBrainTissues.txt",sep="\t")
compiled_post

,nsnps,PP.H0.abf,PP.H1.abf,PP.H2.abf,PP.H3.abf,PP.H4.abf,Tissue,index_variant
0,2015,0.636188,0.098066,0.218374,0.033648,0.013724,Brain_Amygdala,rs141773145
1,2022,0.644494,0.099806,0.209482,0.032427,0.013792,Brain_Anterior_cingulate_cortex_BA24,rs141773145
2,2034,0.013330,0.002080,0.825616,0.128778,0.030196,Brain_Caudate_basal_ganglia,rs141773145
3,2032,0.060078,0.009348,0.779921,0.121331,0.029322,Brain_Cerebellar_Hemisphere,rs141773145
4,2035,0.000059,0.000009,0.839204,0.130955,0.029773,Brain_Cerebellum,rs141773145
5,2035,0.459558,0.071636,0.389932,0.060765,0.018108,Brain_Cortex,rs141773145
6,2028,0.105517,0.016404,0.735825,0.114366,0.027888,Brain_Frontal_Cortex_BA9,rs141773145
7,2035,0.469551,0.073282,0.379477,0.059206,0.018485,Brain_Hippocampus,rs141773145
8,2025,0.375991,0.058281,0.471092,0.073000,0.021637,Brain_Hypothalamus,rs141773145
9,2035,0.183217,0.028599,0.660042,0.103003,0.025139,Brain_Nucleus_accumbens_basal_ganglia,rs141773145


In [166]:
compiled_post_cond = pd.read_csv("Analysis.COLOC_redo/females_only_cond_ApoE4/coloc_posteriors_compiled_allIndexVars_allBrainTissues.txt",sep="\t")
compiled_post_cond

,nsnps,PP.H0.abf,PP.H1.abf,PP.H2.abf,PP.H3.abf,PP.H4.abf,Tissue,index_variant
0,2013,0.627853,0.105706,0.215384,0.036247,0.014810,Brain_Amygdala,rs141773145
1,2022,0.635944,0.107563,0.206703,0.034947,0.014843,Brain_Anterior_cingulate_cortex_BA24,rs141773145
2,2032,0.013149,0.002238,0.814372,0.138566,0.031676,Brain_Caudate_basal_ganglia,rs141773145
3,2030,0.059295,0.010071,0.769760,0.130705,0.030169,Brain_Cerebellar_Hemisphere,rs141773145
4,2033,0.000058,0.000010,0.827426,0.140841,0.031665,Brain_Cerebellum,rs141773145
5,2033,0.453457,0.077141,0.384662,0.065419,0.019321,Brain_Cortex,rs141773145
6,2026,0.104216,0.017674,0.726726,0.123219,0.028165,Brain_Frontal_Cortex_BA9,rs141773145
7,2033,0.463549,0.078907,0.374529,0.063734,0.019280,Brain_Hippocampus,rs141773145
8,2024,0.371126,0.062833,0.464955,0.078697,0.022388,Brain_Hypothalamus,rs141773145
9,2033,0.180811,0.030785,0.651338,0.110870,0.026196,Brain_Nucleus_accumbens_basal_ganglia,rs141773145
